In [2]:
from json import load
import numpy as np
import pandas as pd
import os
import re
from cv2 import imread

In [3]:
with open('annotations/captions_val2014.json') as f:
    ann = load(f)
    f.close()

In [5]:
print(type(ann))
print(ann.keys())
print(type(ann['annotations']))
print(len(ann['annotations']))
print(ann['annotations'][:10])

<class 'dict'>
dict_keys(['info', 'images', 'licenses', 'annotations'])
<class 'list'>
202654
[{'image_id': 203564, 'id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.'}, {'image_id': 179765, 'id': 38, 'caption': 'A black Honda motorcycle parked in front of a garage.'}, {'image_id': 322141, 'id': 49, 'caption': 'A room with blue walls and a white sink and door.'}, {'image_id': 16977, 'id': 89, 'caption': 'A car that seems to be parked illegally behind a legally parked car'}, {'image_id': 106140, 'id': 98, 'caption': 'A large passenger airplane flying through the air.'}, {'image_id': 106140, 'id': 101, 'caption': 'There is a GOL plane taking off in a partly cloudy sky.'}, {'image_id': 322141, 'id': 109, 'caption': 'Blue and white color scheme in a small bathroom.'}, {'image_id': 322141, 'id': 121, 'caption': 'This is a blue and white bathroom with a wall sink and a lifesaver on the wall.'}, {'image_id': 322141, 'id': 163, 'caption': 'A blue boat themed bathroom with

In [6]:
id_caption = {}
for diction in ann['annotations']:
    id_caption[diction['image_id']] = diction['caption']
    

In [7]:
print(list(id_caption.keys())[:10])
print(id_caption[203564])
print(id_caption[179765])
print(id_caption[322141])

[203564, 179765, 322141, 16977, 106140, 571635, 301837, 190236, 315702, 331352]
A clock with the appearance of the wheel of a bicycle 
A motorcycle with its brake extended standing outside
A bathroom with walls that are painted baby blue.


In [8]:
id_caption_pair = {}
for diction in ann['annotations']:
    if diction['image_id'] not in id_caption_pair.keys():
        id_caption_pair[diction['image_id']] = [diction['caption'],]
    else:
        id_caption_pair[diction['image_id']].append(diction['caption'])

In [9]:
print(len(id_caption_pair),'\n')
print(list(id_caption_pair.keys())[:10],"\n")
print(id_caption_pair[203564],'\n')
print(id_caption_pair[179765],'\n')
print(id_caption_pair[322141],'\n')
print(id_caption_pair[16977],'\n')

40504 

[203564, 179765, 322141, 16977, 106140, 571635, 301837, 190236, 315702, 331352] 

['A bicycle replica with a clock as the front wheel.', 'The bike has a clock as a tire.', 'A black metal bicycle with a clock inside the front wheel.', 'A bicycle figurine in which the front wheel is replaced with a clock\n', 'A clock with the appearance of the wheel of a bicycle '] 

['A black Honda motorcycle parked in front of a garage.', 'A Honda motorcycle parked in a grass driveway', 'A black Honda motorcycle with a dark burgundy seat.', 'Ma motorcycle parked on the gravel in front of a garage', 'A motorcycle with its brake extended standing outside'] 

['A room with blue walls and a white sink and door.', 'Blue and white color scheme in a small bathroom.', 'This is a blue and white bathroom with a wall sink and a lifesaver on the wall.', 'A blue boat themed bathroom with a life preserver on the wall', 'A bathroom with walls that are painted baby blue.'] 

['A car that seems to be parked ill

In [10]:
caption_one_pair = {}
indices = []
for key, value in id_caption_pair.items():
    index = np.random.choice(5)
    indices.append(index)
    caption_one_pair[key] = value[index]

In [11]:
print(caption_one_pair[203564])
print(caption_one_pair[179765])
print(caption_one_pair[322141])

A clock with the appearance of the wheel of a bicycle 
A black Honda motorcycle parked in front of a garage.
Blue and white color scheme in a small bathroom.


In [12]:
final_one_pair = {}
total_words = list() 
length = list()
for key,value in caption_one_pair.items():
    temp = re.sub("\.",'',value)
    temp = re.sub("\,",'',temp)
    temp = 'startseq ' + temp + ' endseq'
    temp = ' '.join(list(map(lambda x: x.lower() , temp.split(" ") )))
    length.append(len(temp.split()))
    total_words.extend(temp.split())
    final_one_pair[key] = temp

In [13]:
print(final_one_pair[203564])
print(final_one_pair[179765])
print(final_one_pair[322141],'\n')
print(len(length))
print(length[:20])
max_length = max(length)
print("The maximum length of the sequence is: ",max_length)

startseq a clock with the appearance of the wheel of a bicycle  endseq
startseq a black honda motorcycle parked in front of a garage endseq
startseq blue and white color scheme in a small bathroom endseq 

40504
[13, 12, 11, 15, 14, 10, 10, 10, 11, 12, 19, 10, 11, 12, 12, 10, 21, 10, 11, 10]
The maximum length of the sequence is:  49


In [14]:
print("The total number of words are : ",len(total_words),'\n')
print("Few of the words are as follows: ",total_words[:30],'\n')
vocab = list(set(total_words))
print("The total number of UNIQUE words are : ",len(vocab),'\n')
print("Few of the words are as follows: ", vocab[:30])
vocab_len = len(vocab)

The total number of words are :  503802 

Few of the words are as follows:  ['startseq', 'a', 'clock', 'with', 'the', 'appearance', 'of', 'the', 'wheel', 'of', 'a', 'bicycle', 'endseq', 'startseq', 'a', 'black', 'honda', 'motorcycle', 'parked', 'in', 'front', 'of', 'a', 'garage', 'endseq', 'startseq', 'blue', 'and', 'white', 'color'] 

The total number of UNIQUE words are :  9708 

Few of the words are as follows:  ['steeping', 'reese', 'street', 'duty', 'delicacy', 'amish', 'finding', 'every', 'scottish', 'buns', 'buliding', 'crushed', 'linger', 'gloved', 'bedecked', 'hearth', 'aqua', 'not', 'capsizing', '"no', 'wounds', 'microwaves', 'daytime', 'off-camera', 'attachment', 'highway', 'dingy', 'girl', 'decanter', 'brownies']


In [15]:
i = 1
word_ix = {}
ix_word = {}
for word in vocab:
    word_ix[word] = i
    ix_word[i] = word
    i+=1
print(len(word_ix))
print(len(ix_word))

9708
9708


In [16]:
final_one_num = dict()
indexs = []
for key,value in final_one_pair.items():
    temp = list()
    for word in (value.split()):
        temp.append(word_ix[word])
    final_one_num[key] = temp
    indexs.append(len(temp))
        

In [17]:
print(len(final_one_num))
print(type(final_one_num))
print(list(final_one_num.keys())[:10])
print(final_one_num[203564])
print(final_one_num[179765])
print(final_one_num[322141])

40504
<class 'dict'>
[203564, 179765, 322141, 16977, 106140, 571635, 301837, 190236, 315702, 331352]
[4193, 6647, 3834, 4790, 9369, 8631, 2434, 9369, 2331, 2434, 6647, 3075, 4798]
[4193, 6647, 521, 6766, 4718, 8395, 8488, 5276, 2434, 6647, 4553, 4798]
[4193, 6518, 2394, 2304, 8073, 4629, 8488, 6647, 4931, 7822, 4798]


In [18]:
#import pillow
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array,load_img
from keras.applications import VGG16
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model

Using TensorFlow backend.


In [19]:
model_vgg = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))
model_vgg.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [20]:
def preprocess_image(filename):
    image = load_img(filename, target_size=(224, 224))
    #print(type(image))
    im_array = img_to_array(image)
    image = np.expand_dims(im_array, axis =0)
    image = preprocess_input(image)
    #print(im_array.shape)
    feature = model_vgg.predict(image)
    #print(feature.shape)
    feature = feature.reshape(1,-1)
    #print(feature.shape)
    return feature

In [21]:
image_dir = 'val2014/'
im_name = 'COCO_val2014_000000'
feature_det = {}
for key, value in final_one_num.items():
    if os.path.exists(image_dir+im_name+ str(key) + '.jpg'):
        feature_det[key] = preprocess_image(image_dir+im_name+ str(key) + '.jpg')
    elif os.path.exists(image_dir+im_name+ '0'+ str(key) + '.jpg'):
        feature_det[key] = preprocess_image(image_dir+im_name+ '0'+ str(key) + '.jpg')
    elif os.path.exists(image_dir+im_name+ '00'+ str(key)+ '.jpg'):
        feature_det[key] = preprocess_image(image_dir+im_name+  '00'+ str(key) + '.jpg')
    elif os.path.exists(image_dir+im_name+ '000'+str(key) + '.jpg'):
        feature_det[key] = preprocess_image(image_dir+im_name+ '000'+ str(key) + '.jpg')
    elif os.path.exists(image_dir+im_name+ '0000'+str(key) + '.jpg'):
        feature_det[key] = preprocess_image(image_dir+im_name+ '0000'+ str(key) + '.jpg')
    elif os.path.exists(image_dir+im_name+ '00000'+ str(key) + '.jpg'):
        feature_det[key] = preprocess_image(image_dir+im_name+ '00000'+ str(key) + '.jpg')
    else:
        feature_det[key] = preprocess_image(image_dir+im_name+ '000000'+ str(key) + '.jpg')
        

KeyboardInterrupt: 